# Auto and Cross Correlation within the S&P 500 Returns

## Objective: 
**Determine pairs, lags, correlation(direct & inverse), p-value, and magnitude of relationships between S&P 500 stock returns**
## Approaches:


## Task



In [22]:
# !pip install pandas
# !pip install numpy 
# !pip install matplotlib
# !pip install scipy
# !pip install statsmodels
# !pip install arch

In [23]:
import pandas as pd
import numpy as np
import yfinance as yf 
from matplotlib.colors import ListedColormap


import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

from scipy import signal
from scipy.signal import correlate
from scipy.stats import pearsonr


In [24]:
# Color settings
pink_colors = [
    '#FFC0CB', '#FFB6C1', '#D8BFD8', '#DDA0DD', '#EE82EE', 
    '#DA70D6', '#FF69B4', '#FF1493', '#DB7093', '#C71585'
]
cmap = ListedColormap(pink_colors)

soft_pink = '#FFF1FF'
vivid_pink = '#FCC5FB'
deep_pink = '#FF69B4'
purple = '#FCF1FF'
light_purple = '#FFF9FF'
plum = '#5B095B'

# yass
plt.rcParams.update({
    'font.size': 12,
    'font.family': 'Avenir', 
    'axes.facecolor': light_purple,
    'axes.edgecolor': vivid_pink,
    'axes.labelcolor': vivid_pink,
    'xtick.color': deep_pink,
    'ytick.color': deep_pink,
    'text.color': plum,
    'legend.facecolor': soft_pink,
    'legend.edgecolor': vivid_pink,
    'figure.facecolor': light_purple  
})

### Step 1:  YF Data & DB Data
Import and join datasets of S&P 500 features


In [25]:
sp500list = pd.read_csv('sp500list.txt', header=None)

# Check list length
len(sp500list)

503

In [26]:
def fetch_stock_prices(symbols, start_date, end_date):
    data = yf.download(symbols, start=start_date, end=end_date)
    return data['Adj Close']

symbols = list(sp500list.iloc[:, 0])
start_date, end_date = '2023-01-01', '2024-01-01'
yf_stock_prices = fetch_stock_prices(symbols, start_date, end_date)


[                       0%%                      ]

[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['GEV', 'SOLV']: Exception("%ticker%: Data doesn't exist for startDate = 1672549200, endDate = 1704085200")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-01-01 -> 2024-01-01)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [27]:
db_stock_prices = pd.read_csv("1m_fullyear_equities.csv")

### Step 2: Preprocess and Join Data

**Data Cleaning:**
Clean and preprocess the downloaded data, handling missing values and ensuring all data points are aligned by date.


In [28]:
yf_stock_returns = yf_stock_prices.pct_change()
yf_stock_returns = yf_stock_returns.fillna(0)


/var/folders/jn/h8_6ldvx1wz2fwvbpv9k_nmh0000gn/T/ipykernel_77255/4104894201.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yf_stock_returns = yf_stock_prices.pct_change()


In [29]:
db_stock_prices = db_stock_prices.set_index('ts_event')
db_stock_prices.fillna(method='bfill')

/var/folders/jn/h8_6ldvx1wz2fwvbpv9k_nmh0000gn/T/ipykernel_77255/3282509855.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  db_stock_prices.fillna(method='bfill')


,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
ts_event,,,,,,,,,,,,,,,,,,,,,
2023-05-01 09:30:00+00:00,135.38,13.600,169.220,151.12,119.01,110.52,75.090,280.95,376.81,179.06,...,231.57,29.920,115.770,69.91,116.45,104.075,140.69,138.23,288.00,175.770
2023-05-01 09:31:00+00:00,135.38,13.600,169.220,151.12,119.01,110.52,75.090,280.95,376.81,179.06,...,231.57,29.920,115.770,69.91,116.45,104.075,140.69,138.23,288.00,175.770
2023-05-01 09:32:00+00:00,135.38,13.600,169.220,151.12,119.01,110.52,75.090,280.95,376.81,179.06,...,231.57,29.920,115.770,69.91,116.45,104.075,140.69,138.23,288.00,175.770
2023-05-01 09:33:00+00:00,135.38,13.600,169.220,151.12,119.01,110.52,75.090,280.95,376.81,179.06,...,231.57,29.920,115.770,69.91,116.45,104.075,140.69,138.23,288.00,175.770
2023-05-01 09:34:00+00:00,135.38,13.600,169.220,151.12,119.01,110.52,75.090,280.95,376.81,179.06,...,231.57,29.920,115.770,69.91,116.45,104.075,140.69,138.23,288.00,175.770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30 15:56:00+00:00,138.34,13.700,174.020,162.44,160.12,106.28,92.620,300.97,469.71,202.16,...,252.52,30.555,94.235,53.96,119.05,131.660,142.32,120.80,309.01,159.720
2024-04-30 15:57:00+00:00,138.43,13.695,174.135,162.41,160.16,106.31,92.640,301.01,469.92,202.16,...,252.49,30.560,94.235,54.04,119.11,131.720,142.29,120.80,310.06,159.800
2024-04-30 15:58:00+00:00,138.39,13.690,174.130,162.52,160.16,106.31,93.090,301.01,470.18,202.04,...,252.49,30.585,94.260,54.07,119.11,131.730,142.19,120.86,309.00,159.870


In [30]:
#ensure that the data is in the numeric format
for column in db_stock_prices.columns:
    db_stock_prices[column] = pd.to_numeric(db_stock_prices[column], errors='coerce')


In [31]:
db_stock_returns = db_stock_prices.pct_change()
db_stock_returns = db_stock_returns.fillna(method='bfill')

/var/folders/jn/h8_6ldvx1wz2fwvbpv9k_nmh0000gn/T/ipykernel_77255/3712642574.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  db_stock_returns = db_stock_returns.fillna(method='bfill')


### Step 3: Auto & Cross Correlation

In [34]:
def calculate_correlations_at_lags(data, max_lag=30):
    results = {}
    stocks = data.columns

    # Calculate correlations for each pair of stocks at each lag
    for i in stocks:
        for j in stocks:
            if i != j:
                series1 = data[i]
                series2 = data[j]
                for lag in range(1, max_lag + 1):
                    if lag not in results:
                        results[lag] = []
                    # Shift series2 by 'lag'
                    shifted_series2 = series2.shift(lag)
                    # Drop NA values caused by shifting
                    valid_index = ~np.isnan(shifted_series2)
                    # Calculate correlation and p-value
                    if sum(valid_index) > 0:  # Ensure there are enough data points to calculate correlation
                        corr, p_value = pearsonr(series1[valid_index], shifted_series2[valid_index])
                        results[lag].append({
                            'stock1': i,
                            'stock2': j,
                            'correlation': corr,
                            'p_value': p_value,
                            'lag': lag,
                            'direct': 'direct' if corr > 0 else 'inverse'
                        })

    return results

In [35]:
def pick_best_lag(correlations_at_lags):
    best_lags = {}
    for lags in correlations_at_lags.values():
        for lag in lags:
            stock1, stock2 = lag['stock1'], lag['stock2']
            if (stock1, stock2) not in best_lags:
                best_lags[(stock1, stock2)] = {
                    'correlation': 0,
                    'p_value': 1,
                    'lag': 0,
                    'direct': 'direct'
                }
            if abs(lag['correlation']) > abs(best_lags[(stock1, stock2)]['correlation']):
                best_lags[(stock1, stock2)] = {
                    'correlation': lag['correlation'],
                    'p_value': lag['p_value'],
                    'lag': lag['lag'],
                    'direct': lag['direct']
                }
    return best_lags

In [ ]:
def calculate_selected_lags_correlations(data, selected_lags):
    num_stocks = len(data.columns)
    stocks = data.columns.tolist()
    results = []

    for i in range(num_stocks):
        series1 = data[stocks[i]].fillna(method='ffill').fillna(method='bfill').to_numpy()
        for j in range(i + 1, num_stocks):  # avoid redundancy
            
            series2 = data[stocks[j]].fillna(method='ffill').fillna(method='bfill').to_numpy()

            # Compute full cross-correlation
            full_corr = signal.correlate(series2, series1, mode='full')
            full_lags = signal.correlation_lags(len(series1), len(series2), mode='full')

            # Extract correlations for selected lags
            correlations = []
            lags = []
            for lag in selected_lags:
                index = np.where(full_lags == lag)[0]
                if index.size > 0:
                    correlations.append(full_corr[index[0]])
                    lags.append(full_lags[index[0]])

            results.append({
                'stock1': stocks[i],
                'stock2': stocks[j],
                'correlation': correlations,
                'lags': lags
            })

    return results

db_stock_returns = pd.read_csv("/content/db_stock_returns.csv")
for column in db_stock_returns.columns:
    db_stock_returns[column] = pd.to_numeric(db_stock_returns[column], errors='coerce')

selected_lags = [1, 5, 10, 20, 30, 60, 120, 180]
correlation_results = calculate_selected_lags_correlations(db_stock_returns, selected_lags)


flat_results = []
for entry in correlation_results:
    stock1 = entry['stock1']
    stock2 = entry['stock2']
    correlations = entry['correlation']
    lags = entry['lags']

    # Create a new dict for each correlation and lag
    for corr, lag in zip(correlations, lags):
        flat_results.append({
            'stock1': stock1,
            'stock2': stock2,
            'correlation': corr,
            'lag': lag
        })


In [ ]:
from scipy.stats import pearsonr
import numpy as np

def calculate_correlations(lag_df):
    results = []
    for i in range(len(lag_df)):
        for j in range(i+1, len(lag_df)):  # Avoid duplicate pairs and self-correlation
            specific_lag = lag_df['lag'].iloc[i]  # Get the specific lag for the current row
            series1 = lag_df.iloc[:, i]
            series2 = lag_df.iloc[:, j].shift(specific_lag)
            valid_index = ~np.isnan(series2)
            if sum(valid_index) > 0:  # Ensure there are enough data points to calculate correlation
                corr, p_value = pearsonr(series1[valid_index], series2[valid_index])
                results.append({
                    'stock1': lag_df.columns[i],
                    'stock2': lag_df.columns[j],
                    'correlation': corr,
                    'p_value': p_value,
                    'lag': specific_lag,
                    'direct': 'direct' if corr > 0 else 'inverse'
                })
    return results

In [ ]:
correlations_at_lags = calculate_correlations_at_lags(stock_returns, max_lag=30)
best_lags = pick_best_lag(correlations_at_lags)
best_lags = pd.DataFrame(best_lags).T
best_lags['r_squared'] = best_lags['correlation'] ** 2
best_lags.sort_values('r_squared', ascending= False)